# 🛡️ 詐欺交易預測 Demo：Flask API + 模擬資料集

本教學示範如何：
1. 建立簡易 Flask API 接收交易資料
2. 載入訓練好的機器學習模型進行預測
3. 使用 `ngrok` 讓本地 API 可於 Colab 測試

📌 適合展示如何將模型應用於實務場景

In [49]:
# ✅ 安裝必要套件
!pip install flask-ngrok scikit-learn pandas numpy --quiet

In [ ]:
# 📦 建立模型並儲存
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import joblib

# 模擬資料集
n = 5000
locations = ['台北', '台中', '高雄']
devices = ['mobile', 'desktop']

df = pd.DataFrame({
    'amount': np.random.exponential(scale=1000, size=n).round(2),
    'location': np.random.choice(locations, n),
    'device_type': np.random.choice(devices, n),
    'time': np.random.uniform(0, 24, n).round(2),
    'is_fraud': np.random.choice([0, 1], p=[0.98, 0.02], size=n)
})

# 特徵工程
X = pd.get_dummies(df.drop('is_fraud', axis=1), drop_first=True)
y = df['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

model = LogisticRegression(class_weight='balanced')
model.fit(X_train_scaled, y_train)

# 儲存模型與 scaler
joblib.dump(model, 'fraud_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
# 🖥️ 建立 Flask API 並進行測試
from flask import Flask, request, jsonify
import joblib
import pandas as pd
from threading import Thread
import time
import requests
from google.colab.output import eval_js
from pyngrok import ngrok

# 載入模型與 scaler
model = joblib.load('fraud_model.pkl')
scaler = joblib.load('scaler.pkl')

# --- Flask App ---
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    df = pd.DataFrame([data])

    # 進行 one-hot encoding
    df_encoded = pd.get_dummies(df, columns=['location', 'device_type'], drop_first=True)

    # 補上訓練時有，但此次請求沒有的欄位
    for col in X_train.columns: # 使用訓練時的欄位來對齊
        if col not in df_encoded.columns:
            df_encoded[col] = 0

    # 確保欄位順序與訓練時一致
    df_encoded = df_encoded[X_train.columns]

    X_scaled = scaler.transform(df_encoded)
    pred = model.predict(X_scaled)[0]
    prob = model.predict_proba(X_scaled)[0][1]

    return jsonify({
        "is_fraud": int(pred),
        "fraud_probability": round(prob, 4)
    })

def run_app():
    # 不使用 run_with_ngrok，改為手動啟動
    app.run(port=5000)

# --- Ngrok and Testing ---
# 提示使用者輸入 ngrok authtoken
authtoken = input("請貼上您的 ngrok authtoken: ")
ngrok.set_auth_token(authtoken)

# 在背景執行 Flask app
thread = Thread(target=run_app)
thread.daemon = True
thread.start()
time.sleep(3) # 等待 app 啟動

# 建立 ngrok 通道並取得 URL
tunnel = ngrok.connect(5000)
public_url = tunnel.public_url
print(f"✅ Ngrok public URL: {public_url}")


# --- 測試 API ---
sample = {
    "amount": 5000,
    "location": "台中",
    "device_type": "mobile",
    "time": 2.5
}

try:
    res = requests.post(f"{public_url}/predict", json=sample)
    res.raise_for_status() # 如果請求失敗，會拋出例外
    print("\n✅ 測試請求成功！")
    print("API 回應:", res.json())
except requests.exceptions.RequestException as e:
    print(f"\n❌ 測試請求失敗: {e}")
finally:
    # 關閉 ngrok 連線
    ngrok.kill()

### ✅ 測試 API
使用 `requests` 套件傳送 POST 請求：

In [ ]:
import requests

url = "http://localhost:5000/predict"
sample = {
    "amount": 5000,
    "location": "台中",
    "device_type": "mobile",
    "time": 2.5
}

res = requests.post(url, json=sample)
print(res.json())